In [1]:
import polars as pl
import matplotlib.pyplot as plt

pl.Config.set_tbl_rows(6)

polars.config.Config

In [2]:
df = pl.read_parquet('ushcn_monthly_data.parquet')
df

coop_id,year,month,element,dataset_type,value,dmflag,qcflag,dsflag
str,u16,u8,str,str,f64,str,str,str
"""231037""",1893,1,"""tmax""","""raw""",-0.24,null,null,null
"""231037""",1893,2,"""tmax""","""raw""",3.15,null,null,null
"""231037""",1893,3,"""tmax""","""raw""",11.96,"""b""",null,null
…,…,…,…,…,…,…,…,…
"""189440""",2025,10,"""tmax""","""FLs.52j""",null,null,null,null
"""189440""",2025,11,"""tmax""","""FLs.52j""",null,null,null,null
"""189440""",2025,12,"""tmax""","""FLs.52j""",null,null,null,null


In [3]:
def clean_source(df, dataset_type):
    return (
        df
        .filter(
            pl.col('element') == 'tmax',
            pl.col('dataset_type') == dataset_type,
        )
        .with_columns(
            pl.col('value').alias('temp_c'),
            (pl.col('value') * 9/5 + 32).alias('temp_f')
        )
        .drop(['element', 'dataset_type', 'value'])
    )

In [4]:
raw = clean_source(df, 'raw')

In [5]:
altered = clean_source(df, 'FLs.52j')

In [6]:
combined = (
    altered
    .join(
        raw,
        on=['coop_id', 'year', 'month'],
        how='left',
        suffix='_raw'
    )
    .sort(['year', 'month', 'coop_id'])
    .select(['year', 'month', 'coop_id', 'temp_f', 'dmflag', 'temp_f_raw', 'dmflag_raw'])
    .with_columns(
        (pl.col('temp_f') - pl.col('temp_f_raw')).alias('adjustment')
    )
)
combined[100000:]

year,month,coop_id,temp_f,dmflag,temp_f_raw,dmflag_raw,adjustment
u16,u8,str,f64,str,f64,str,f64
1898,3,"""344235""",62.51,"""E""",null,null,null
1898,3,"""344298""",60.998,"""E""",null,null,null
1898,3,"""344573""",59.306,"""a""",64.004,"""a""",-4.698
…,…,…,…,…,…,…,…
2025,12,"""489615""",null,null,null,null,null
2025,12,"""489770""",null,null,null,null,null
2025,12,"""489905""",null,null,null,null,null


In [7]:
(
    combined
    .filter(
        pl.col('month') == 7,
        pl.col('year') == 1868
    )
)

year,month,coop_id,temp_f,dmflag,temp_f_raw,dmflag_raw,adjustment
u16,u8,str,f64,str,f64,str,f64
1868,7,"""144559""",null,null,null,null,null


In [ ]:
adjustments = (
    altered
    .join(
        raw,
        on=['coop_id', 'year', 'month'],
        how='left',
        suffix='_raw'
    )
    .filter(
        pl.col('temp_c').is_not_null(),
        pl.col('temp_c_raw').is_not_null(),
    )
    .sort(['year', 'month', 'coop_id'])
    .select(['year', 'month', 'coop_id', 'temp_f', 'dmflag', 'temp_f_raw', 'dmflag_raw'])
    .with_columns(
        (pl.col('temp_f') - pl.col('temp_f_raw')).alias('adjustment')
    )
)

In [22]:
stats = (
    combined
    .filter(
        pl.col('month') == 7
    )
    .group_by('year')
    .agg(
        pl.col('temp_f').is_not_null().sum().alias('n_obs'),
        pl.col('adjustment').filter(pl.col('adjustment') != 0).count().alias('n_adjustments'),
        pl.col('temp_f').filter((pl.col('dmflag') == 'E') & (pl.col('temp_f').is_not_null())).count().alias('n_estimates')
    #     pl.count('adjustment').alias('n_opportunities'),
    #     pl.col('adjustment').filter(pl.col('adjustment') != 0).count().alias('n_adjustments'),
    #     pl.mean('adjustment').alias('avg_adjustment'),
    #     pl.col('temp_f').filter(pl.col('dmflag') == 'E').count().alias('n_estimates'),
    #     pl.col('temp_f').filter(pl.col('dmflag') == 'E').mean().alias('avg_estimate'),
    #     pl.col('temp_f').filter((pl.col('dmflag') != 'E') | pl.col('dmflag').is_null()).mean().alias('avg_not_estimate'),
    )
    # .with_columns(
    #     (100 * pl.col('n_adjustments') / pl.col('n_opportunities')).alias('percent_adjusted')
    # )
)
stats

year,n_obs,n_adjustments,n_estimates
u16,u32,u32,u32
1868,0,0,0
1869,7,2,5
1870,6,1,5
…,…,…,…
2023,1218,658,504
2024,1216,661,541
2025,0,0,0


In [ ]:
adj_stats&['n_obs'].sum()

In [ ]:

adj_stats['n_opportunities'].sum()

In [ ]:
def make_chart(line_field):
    # Create the figure
    plt.figure(figsize=(9, 5))

    line = plt.plot(adj_stats['year'], adj_stats[line_field], color='#6f0d02', linewidth=2, label=line_field)[0]
    
    # Set labels and title for the left y-axis
    plt.xlabel('Year')
    plt.ylabel('%', color='#6f0d02')
    plt.tick_params(axis='y', labelcolor='#6f0d02')
    plt.title('Dude')
    plt.grid(True)

    # Create a second y-axis for the bar plot
    ax2 = plt.twinx()
    # Plot bars for count_non_zero
    bars = ax2.bar(adj_stats['year'], adj_stats['n_opportunities'], color='#1f77b4', alpha=0.5, label='# Opportunities', width=0.4)

    # Set labels for the right y-axis
    ax2.set_ylabel('Count', color='#1f77b4')
    ax2.tick_params(axis='y', labelcolor='#1f77b4')

    # Combine legends from both axes
    plt.legend([line, bars], [line_field, '# Opportunities'], loc='upper left', bbox_to_anchor=(0.0, 0.85))

    # Adjust layout
    plt.tight_layout()

    # Show plot
    plt.show()

In [ ]:
make_chart('percent_adjusted')

In [ ]:
make_chart('avg_adjustment')